# Apply the DIAMOnD algorithm to prioritize candidates for every Chromosome

In [1]:
# Load the required packages
import pandas as pd
import os, json
from itertools import product
import networkx as nx
import scipy.stats
from collections import defaultdict
import numpy as np
import sklearn.metrics
from datetime import datetime

In [2]:
def get_neighbors_and_degrees(G):

    neighbors,all_degrees = {},{}
    for node in G.nodes():
        nn = set(G.neighbors(node))
        neighbors[node] = nn
        all_degrees[node] = G.degree(node)

    return neighbors,all_degrees

In [3]:
def compute_all_gamma_ln(N):
    """
    precomputes all logarithmic gammas
    """
    gamma_ln = {}
    for i in range(1,N+1):
        gamma_ln[i] = scipy.special.gammaln(i)

    return gamma_ln

In [4]:
def reduce_not_in_cluster_nodes(all_degrees,neighbors,G,not_in_cluster,cluster_nodes,alpha):
    reduced_not_in_cluster = {}
    kb2k = defaultdict(dict)
    for node in not_in_cluster:

        k = all_degrees[node]
        kb = 0
        # Going through all neighbors and counting the number of module neighbors
        for neighbor in neighbors[node]:
            if neighbor in cluster_nodes:
                kb += 1

        #adding wights to the the edges connected to seeds
        k += (alpha-1)*kb
        kb += (alpha-1)*kb
        kb2k[kb][k] =node

    # Going to choose the node with largest kb, given k
    k2kb = defaultdict(dict)
    for kb,k2node in kb2k.items():
        min_k = min(k2node.keys())
        node = k2node[min_k]
        k2kb[min_k][kb] = node

    for k,kb2node in k2kb.items():
        max_kb = max(kb2node.keys())
        node = kb2node[max_kb]
        reduced_not_in_cluster[node] =(max_kb,k)

    return reduced_not_in_cluster

In [5]:
def pvalue(kb, k, N, s, gamma_ln):
    """
    -------------------------------------------------------------------
    Computes the p-value for a node that has kb out of k links to
    seeds, given that there's a total of s sees in a network of N nodes.

    p-val = \sum_{n=kb}^{k} HypergemetricPDF(n,k,N,s)
    -------------------------------------------------------------------
    """
    p = 0.0
    for n in range(kb,k+1):
        if n > s:
            break
        prob = gauss_hypergeom(n, s, N-s, k, gamma_ln)
        # print prob
        p += prob

    if p > 1:
        return 1
    else:
        return p

In [6]:
def gauss_hypergeom(x, r, b, n, gamma_ln):
    return np.exp(logchoose(r, x, gamma_ln) +
                  logchoose(b, n-x, gamma_ln) -
                  logchoose(r+b, n, gamma_ln))

In [7]:
def logchoose(n, k, gamma_ln):
    if n-k+1 <= 0:
        return scipy.infty
    lgn1  = gamma_ln[n+1]
    lgk1  = gamma_ln[k+1]
    lgnk1 = gamma_ln[n-k+1]
    return lgn1 - [lgnk1 + lgk1]

In [8]:
def diamond_iteration_of_first_X_nodes(G,S,X,alpha):

    """

    Parameters:
    ----------
    - G:     graph
    - S:     seeds
    - X:     the number of iterations, i.e only the first X gened will be
             pulled in
    - alpha: seeds weight

    Returns:
    --------

    - added_nodes: ordered list of nodes in the order by which they
      are agglomerated. Each entry has 4 info:

      * name : dito
      * k    : degree of the node
      * kb   : number of +1 neighbors
      * p    : p-value at agglomeration

    """

    N = G.number_of_nodes()

    added_nodes = []


    # ------------------------------------------------------------------
    # Setting up dictionaries with all neighbor lists
    # and all degrees
    # ------------------------------------------------------------------
    neighbors,all_degrees = get_neighbors_and_degrees(G)

    # ------------------------------------------------------------------
    # Setting up initial set of nodes in cluster
    # ------------------------------------------------------------------

    cluster_nodes = set(S)
    not_in_cluster = set()
    s0 = len(cluster_nodes)

    s0 += (alpha-1)*s0
    N +=(alpha-1)*s0

    # ------------------------------------------------------------------
    # precompute the logarithmic gamma functions
    # ------------------------------------------------------------------
    gamma_ln = compute_all_gamma_ln(N+1)

    # ------------------------------------------------------------------
    # Setting initial set of nodes not in cluster
    # ------------------------------------------------------------------
    for node in cluster_nodes:
        not_in_cluster |= neighbors[node]
    not_in_cluster -= cluster_nodes


    # ------------------------------------------------------------------
    #
    # M A I N     L O O P
    #
    # ------------------------------------------------------------------

    all_p = {}

    while len(added_nodes) < X:

        # ------------------------------------------------------------------
        #
        # Going through all nodes that are not in the cluster yet and
        # record k, kb and p
        #
        # ------------------------------------------------------------------

        info = {}

        pmin = 10
        next_node = 'nix'
        reduced_not_in_cluster = reduce_not_in_cluster_nodes(all_degrees,
                                                             neighbors,G,
                                                             not_in_cluster,
                                                             cluster_nodes,alpha)

        for node,kbk in reduced_not_in_cluster.items():
            # Getting the p-value of this kb,k
            # combination and save it in all_p, so computing it only once!
            kb,k = kbk
            try:
                p = all_p[(k,kb,s0)]
            except KeyError:
                p = pvalue(kb, k, N, s0, gamma_ln)
                all_p[(k,kb,s0)] = p

            # recording the node with smallest p-value
            if p < pmin:
                pmin = p
                next_node = node

            info[node] = (k,kb,p[0])

        # ---------------------------------------------------------------------
        # Adding node with smallest p-value to the list of aaglomerated nodes
        # ---------------------------------------------------------------------
        added_nodes.append((next_node,
                            info[next_node][0],
                            info[next_node][1],
                            info[next_node][2]))

        # Updating the list of cluster nodes and s0
        cluster_nodes.add(next_node)
        s0 = len(cluster_nodes)
        not_in_cluster |= ( neighbors[next_node] - cluster_nodes )
        not_in_cluster.remove(next_node)

    return added_nodes

In [9]:
# Read in the edgelist to create the network
G = nx.read_edgelist("/Users/vlietstraw/git/Post-GWAS/unfiltered_protein_protein_interactions.csv", delimiter = ",", nodetype = int)

In [10]:
# Load the mapping file
with open("/Users/vlietstraw/git/Post-GWAS/ENSEMBL_mappings.json", "r") as fp:
    ensembl_dict = json.load(fp)

In [11]:
#all_bp_distances = [25, 50, 100, 500, 1000, 2000]
all_bp_distances = [25, 50, 100, 500, 1000, 2000, "depict"]
refsets = ["Teslovich"]#, "DeRycke", "farashi", "farashi p-value cutoff"]

all_metrics = pd.DataFrame(list(product(refsets, all_bp_distances)), columns = ["refset", "bp distance"])

In [ ]:
nCandidates = 1000
    
for am_index, am_values in all_metrics.iterrows():
    outcomes = pd.DataFrame()
    outcomes2 = pd.DataFrame()
    
    print("Predicting row " + str(am_index + 1) + " of " + str(len(all_metrics)))
    if am_values["refset"] == "farashi":
        ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/Farashi/Farashi full 2000000 bp distance no pvalue filtering.csv")

    if am_values["refset"] == "farashi p-value cutoff":
        ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/Farashi/Farashi full 2000000 bp distance no pvalue filtering.csv")
        ref = ref[ref["GWAS/eQTL p-value¥"] <= float("5e-8")]

    if am_values["refset"] == "DeRycke":
        ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/DeRycke/DeRycke reference set.csv")
        ref.columns = ["SNP ID", "chromosome", "location", "gene_ids", "gene name", "gene start", "gene stop", "Diff expression", "Class", "bp distance absolute", "bp distance", "Gene rank"]

    if am_values["refset"] == "Teslovich":
        ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/Teslovich/Teslovich reference set.csv")
        ref.columns = ["SNP ID", "chromosome", "location", "P", "gene_ids", "gene name", "gene start", "gene stop", "Class", "bp distance absolute", "bp distance", "Gene rank"]

    ref["nodeID"] = [ensembl_dict[x] if x in ensembl_dict.keys() else None for x in ref["gene_ids"]]

    # Set bp distance cutoff
    if am_values["bp distance"] != "depict":
        max_bp_distance = am_values["bp distance"]
        max_bp_distance = max_bp_distance * 1000
        ref = ref[ref["bp distance absolute"] <= max_bp_distance]
    elif am_values["bp distance"] == "depict":
        depict = pd.read_csv("~/git/DEPICT/outcomes/Teslovich for paper Wytze/Teslovich_output_geneprioritization.txt", sep = "\t")
        depict["nodeID"] = [ensembl_dict[x] if x in ensembl_dict.keys() else None for x in depict["Ensembl gene ID"]]
        
        depict["Locus"] = depict["Locus"].astype(str).apply(lambda x: x.split(";"))
        depict = depict.explode("Locus")

        snp_replacement_dict = {"rs113645266" : "rs6557271",
                        "rs150282463" : "rs13137700",
                        "rs67276543" : "rs34884832"}
        depict["Locus"] = depict["Locus"].replace(snp_replacement_dict)

        depict = depict[["Locus", "nodeID"]]
        depict.columns = ["SNP ID", "nodeID"]

        ref = ref.merge(depict, on = ["SNP ID", "nodeID"], how = "inner")

    # Drop all unmappable candidates
    ref.dropna(subset = ["nodeID"], inplace = True)
    ref["nodeID"] = ref["nodeID"].astype(int)

    # Drop all SNPs which no longer have a positive case
    pos_counts = ref.groupby("SNP ID")["Class"].sum()
    ref = ref[~ref["SNP ID"].isin(pos_counts[pos_counts == 0].index)]
    
#    SNPs = list(set(ref["SNP ID"]))
#    for snp in SNPs:
#        print("Predicting candidates for " + snp + ", number " + str(SNPs.index(snp) + 1) + " out of " + str(len(SNPs)))
#        out = pd.DataFrame(diamond_iteration_of_first_X_nodes(G, ref[(ref["SNP ID"] != snp) & (ref["Class"] == 1)]["nodeID"], nCandidates, 1), 
#                   columns = ["nodeID", "unknown1", "unknown2", "predicted"])
#        out["SNP ID"] = snp
#        out = out[out["nodeID"].isin(ref["nodeID"][ref["SNP ID"] == snp])]
#        out["Class"] = [1 if x in list(ref["nodeID"][(ref["Class"] == 1) & (ref["SNP ID"] == snp)]) else 0 for x in out["nodeID"]]
#        outcomes = outcomes.append(out) # predicted is a p-value, so lower is better
       

    
    chromosomes = list(set(ref["chromosome"]))
    for chrom in chromosomes:
        print("Getting candidates for chromosome " + str(chrom))
        # Write away the seed to an input file
        out = pd.DataFrame(diamond_iteration_of_first_X_nodes(G, ref[(ref["chromosome"] != chrom) & (ref["Class"] == 1)]["nodeID"], nCandidates, 1), 
                           columns = ["nodeID", "unknown1", "unknown2", "predicted"]) # predicted is a p-value, so lower is better
        out["chromosome"] = chrom
        out["Class"] = [1 if x in list(ref["nodeID"][ref["Class"] == 1]) else 0 for x in out["nodeID"]]
        out = out[out["nodeID"].isin(ref["nodeID"][ref["chromosome"] == chrom])]
        outcomes2 = outcomes2.append(out)
        
  #  outcomes = outcomes.sort_values(["SNP ID", "predicted"], ascending = True)
  #  outcomes["For-SNP rank"] = outcomes.groupby("SNP ID").cumcount() + 1

  #  all_metrics.at[am_index, "Recall snps"] = len(set(outcomes["SNP ID"]))
  #  all_metrics.at[am_index, "Recall genes"] = sum(outcomes["Class"])
        
  #  fpr, tpr, thresholds = sklearn.metrics.roc_curve(outcomes["Class"], -outcomes["For-SNP rank"], pos_label = 1)
  #  all_metrics.at[am_index, "ROC-AUC overall (lso)"] = sklearn.metrics.auc(fpr, tpr) * 100

#    # Calculate the ROC-AUC for every SNP and average the result
#    SNPS2 = list(set(outcomes["SNP ID"]))
#    aucs = []
#    for snp in SNPS2:
#        if len(set(outcomes["Class"][outcomes["SNP ID"] == snp])) == 1:
#            aucs.append(list(set(outcomes["Class"][outcomes["SNP ID"] == snp]))[0])
#        else:
#            fpr, tpr, thresholds = sklearn.metrics.roc_curve(outcomes["Class"][outcomes["SNP ID"] == snp], -outcomes["For-SNP rank"][outcomes["SNP ID"] == snp], pos_label = 1)
#            aucs.append(sklearn.metrics.auc(fpr, tpr))
#    all_metrics.at[am_index, "ROC-AUC - mean per snpl (lso)"] = sum(aucs)/len(aucs)


    # In[22]:


    # Calculate hits @1
#    all_metrics.at[am_index, "Hits@1(lso)"] = sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] == 1)])


    # In[23]:


    # Calculate hits @3
#    all_metrics.at[am_index, "Hits@3(lso)"] = sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] <= 3)])


    # In[24]:


    # Calculate hits @5
#    all_metrics.at[am_index, "Hits@5(lso)"] = sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] <= 5)])


    # In[25]:


    # Calculate hits @10
#    all_metrics.at[am_index, "Hits@10(lso)"] = sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] <= 10)])


    # In[26]:


#    all_metrics.at[am_index, "Mean rank (lso)"] = outcomes["For-SNP rank"][(outcomes["Class"] == 1)].mean()


    # In[27]:


#    all_metrics.at[am_index, "Median rank (lso)"] = outcomes["For-SNP rank"][outcomes["Class"] == 1].quantile(q = [0,0.25,0.5,0.75,1])[0.50]


    # ## Evaluate leave-chromosome-out

    # In[28]:


    outcomes2 = outcomes2.sort_values(["chromosome", "predicted"])
    outcomes2["For-chromosome rank"] = outcomes2.groupby("chromosome").cumcount() + 1


    # In[29]:


    #chromosomes = list(set(outcomes2["chromosome"]))
    #aucs = []
    #for chrom in chromosomes:
    #  fpr, tpr, thresholds = sklearn.metrics.roc_curve(outcomes2["Class"][outcomes2["chromosome"] == chrom], -outcomes2["For-chromosome rank"][outcomes2["chromosome"] == chrom], pos_label = 1)
    #  aucs.append(sklearn.metrics.auc(fpr, tpr))
    #print(sum(aucs)/len(aucs))


    # In[30]:


    ref = ref.merge(outcomes2[["nodeID", "predicted"]], on = "nodeID")#, how = "left")
    pos_counts = ref.groupby("SNP ID")["Class"].sum()
    ref = ref[~ref["SNP ID"].isin(pos_counts[pos_counts == 0].index)]
    
    if len(ref) > 0:
        all_metrics.at[am_index, "Recall snps (lco)"] = len(set(ref["SNP ID"]))
        all_metrics.at[am_index, "Recall genes (lco)"] = sum(ref["Class"])

        # In[31]:


        ref = ref.sort_values(["SNP ID", "predicted"], ascending = True)
        ref["For-SNP rank"] = ref.groupby("SNP ID").cumcount() + 1


        # In[32]:


        fpr, tpr, thresholds = sklearn.metrics.roc_curve(ref["Class"], -ref["For-SNP rank"], pos_label = 1)
        all_metrics.at[am_index, "ROC-AUC overall (lco)"] = sklearn.metrics.auc(fpr, tpr) * 100


        # In[33]:


        # Calculate the ROC-AUC for every SNP and average the result
        SNPS2 = list(set(ref["SNP ID"]))
        aucs = []
        for snp in SNPS2:
          if len(set(ref["Class"][ref["SNP ID"] == snp])) == 1:
              aucs.append(list(set(ref["Class"][ref["SNP ID"] == snp]))[0])
          else:
              fpr, tpr, thresholds = sklearn.metrics.roc_curve(ref["Class"][ref["SNP ID"] == snp], -ref["For-SNP rank"][ref["SNP ID"] == snp], pos_label = 1)
              aucs.append(sklearn.metrics.auc(fpr, tpr))
        all_metrics.at[am_index, "ROC-AUC - mean per snpl (lco)"] = sum(aucs)/len(aucs)


        # In[34]:


        # Calculate hits @1
        all_metrics.at[am_index, "Hits@1(lco)"] = sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] == 1)])


        # In[35]:


        # Calculate hits @3
        all_metrics.at[am_index, "Hits@3(lco)"] = sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] <= 3)])


        # In[36]:


        # Calculate hits @5
        all_metrics.at[am_index, "Hits@5(lco)"] = sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] <= 5)])


        # In[37]:


        # Calculate hits @10
        all_metrics.at[am_index, "Hits@10(lco)"] = sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] <= 10)])


        # In[38]:


        all_metrics.at[am_index, "Mean rank (lco)"] = ref["For-SNP rank"][(ref["Class"] == 1)].mean()


        # In[39]:


        all_metrics.at[am_index, "Median rank (lco)"] = ref["For-SNP rank"][ref["Class"] == 1].quantile(q = [0,0.25,0.5,0.75,1])[.50]
    
    outcomes2.to_csv("/Users/vlietstraw/git/Post-GWAS/DIAMOND/" + am_values["refset"] + " diamond predictions with bp distance " + str(am_values["bp distance"]) + ".csv", sep = ";", decimal = ",", index = False)

Predicting row 1 of 7
Getting candidates for chromosome 16
Getting candidates for chromosome 1
Getting candidates for chromosome 6


In [ ]:
all_metrics.to_csv("/Users/vlietstraw/git/Post-GWAS/DIAMOND/all_variations_performance_metrics " + am_values["refset"] + " " + datetime.today().strftime("%d-%m-%Y") +".csv", sep = ";", decimal = ",", index = False)